# ======================================================
# Author: Nika Lu Yang
# Date: April 2025
# Bachelor Thesis Research (BTR): "Unraveling the Impact of Aging on Parkinson’s Disease-derived Neuroepithelial Stem Cells"
# Maastricht Science Programme (MSP)
# ======================================================

# Requirements

In [ ]:
import cv2
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import tifffile  
from skimage import morphology, measure
from skimage.color import gray2rgb
from datetime import datetime
from skimage.io import imsave
from skimage.measure import perimeter as sk_perimeter, find_contours

# Configurable Parameters

In [ ]:
# Input Folder -> where the images are located
# Output Folder -> where the results will be saved

input_folder =  "/Users/nikalu/Downloads/Images_Organoids"
output_folder = "/Users/nikalu/Downloads/Organoids_Analysis_Results"

params = {
    "high_int": 255,
    "low_int": 20,
    "small": 300,
    "perimeter_max": 8000,
    "ecc_max": 0.8
}

# Initiziales Files

In [ ]:
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
save_path = os.path.join(output_folder, f'Analysis_{timestamp}')
os.makedirs(save_path, exist_ok=True)

input_root = input_folder
output_root = save_path
loaded_images = []

for date_folder in os.listdir(input_root):
    date_path = os.path.join(input_root, date_folder)
    if not os.path.isdir(date_path):
        continue  # Skip files

    # Loop over all type folders in this date folder
    for type_folder in os.listdir(date_path):
        type_path = os.path.join(date_path, type_folder)
        if not os.path.isdir(type_path):
            continue  # Skip files

        print(f"\n--- Processing: {date_folder}/{type_folder} ---")

        # Create output folder in the same structure
        results_base = os.path.join(output_root, date_folder, type_folder)
        os.makedirs(results_base, exist_ok=True)

        # Create subfolders for raw, mask, rp, failed
        preview_paths = {
            'raw': os.path.join(results_base, 'Raw'),
            'mask': os.path.join(results_base, 'Mask'),
            'rp': os.path.join(results_base, 'Rp'),
            'failed': os.path.join(results_base, 'Failed')
        }
        for p in preview_paths.values():
            os.makedirs(p, exist_ok=True)

        # List for measurements
        results_list = []

        # Check if images are directly in type_path or in its subfolders
        subdirs = [os.path.join(type_path, d) for d in os.listdir(type_path) if os.path.isdir(os.path.join(type_path, d))]

        if subdirs:
            # Images are in subfolders (e.g., 20250208 organoids style)
            for subdir in subdirs:
                # Create matching subfolders in the output paths:
                preview_subfolders = {
                    key: os.path.join(p, os.path.relpath(subdir, type_path)) for key, p in preview_paths.items()
                }
                for pp in preview_subfolders.values():
                    os.makedirs(pp, exist_ok=True)

                for filename in os.listdir(subdir):
                    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tif', '.tiff')):
                        image_path = os.path.join(subdir, filename)
                        raw_img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
                        print(f"Loaded image: {image_path}")
                        loaded_images.append((image_path, raw_img, preview_subfolders))
        else:
            # Images are directly in this folder (e.g., P7_02-04 style)
            for filename in os.listdir(type_path):
                if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tif', '.tiff')):
                    image_path = os.path.join(type_path, filename)
                    raw_img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
                    loaded_images.append((image_path, raw_img, preview_paths))
                    print(f"Loaded image: {image_path}")




# Process Images

In [ ]:
def size_qc(image_path, raw_image, paths, high_int, low_int, small, perimeter_max, ecc_max):  # size quality control
    # Get pixel size from TIFF metadata (default to 1.0 µm/pixel if fails)
    pixel_size = 1.0
    try:
        with tifffile.TiffFile(image_path) as tif:
            metadata = tif.pages[0].tags
            x_res = metadata["XResolution"].value
            y_res = metadata["YResolution"].value
            res_unit = metadata["ResolutionUnit"].value  # 3 = centimeter

            x_pixels_per_cm = x_res[0] / x_res[1]
            y_pixels_per_cm = y_res[0] / y_res[1]
            pixel_size_x = (1 / x_pixels_per_cm) * 10_000  # µm/pixel
            pixel_size_y = (1 / y_pixels_per_cm) * 10_000
            pixel_size = (pixel_size_x + pixel_size_y) / 2
    except Exception as e:
        print(f"[Warning] Could not extract pixel size from metadata: {e}")
        print("→ Using default 1.0 µm/pixel")
   
    # Process image
    blurred = cv2.GaussianBlur(raw_image, (15, 15), 16) # Gaussian blur to reduce noise and smooth the image. (sigma more >20 not good)
    """
    blurred = cv2.medianBlur(raw_image, 25) # Medianblur takes the median of all the pixels under the kernel area and the central element is replaced with this median value
    """
    _, mask = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU) # Binarizes the image using Otsu's method + inverted threshold [separates objects (dark) from the background (bright)]
    mask = mask > 0 # Converts binary mask from 0/255 to True/False for logical operations (boolean mask)
    mask = morphology.remove_small_objects(mask, small) # Removes small objects below specified size threshold
    mask = morphology.remove_small_holes(mask, area_threshold=1000) # Fills in small holes <1000 pixels inside objects to create cleaner shapes

    # Debug view of raw mask
    plt.imshow(mask, cmap='gray')
    plt.title("Raw Binary Mask")
    plt.axis('off')
    # plt.show()

    #Analyze labeled regions
    labels = measure.label(mask) # Labels each connected object in the mask.
    regions = measure.regionprops(labels, intensity_image=raw_image) # Extracts region properties (area, perimeter, eccentricity) using the original grayscale image
    valid_regions = [r for r in regions if r.perimeter < perimeter_max and r.eccentricity < ecc_max] # Filter based on shape criteria
    if not valid_regions: # safety check and fallback in case no regions pass the quality control filters (perimeter_max + ecc_max).
        print(f"No valid objects found in {os.path.basename(image_path)}.")
        _save_image(raw_image, os.path.join(paths['failed'], os.path.basename(image_path)))
        return None

    image_center = np.array(raw_image.shape) / 2
    def dist_to_center(r):
        return np.linalg.norm(np.array(r.centroid) - image_center)
    regions = sorted(regions, key=lambda r: (-r.area, dist_to_center(r)))

    # Select region with largest pixel area 
    largest_obj = regions[0] # select largest object based on the number of pixels
    mask_selected = (labels == largest_obj.label) # Creates a mask that only contains the selected object

    # Convert to µm
    true_area_px = np.sum(mask_selected)
    true_perimeter_px = sk_perimeter(mask_selected)
    major_axis_px = largest_obj.major_axis_length

    true_area_um2 = true_area_px * (pixel_size ** 2)
    true_perimeter_um = true_perimeter_px * pixel_size
    major_axis_um = major_axis_px * pixel_size

    # Convert from pixels to µm/µm²
    true_area_px = np.sum(mask_selected)
    true_perimeter_px = sk_perimeter(mask_selected)
    major_axis_px = largest_obj.major_axis_length

    true_area_um2 = true_area_px * (pixel_size ** 2)
    true_perimeter_um = true_perimeter_px * pixel_size
    major_axis_um = major_axis_px * pixel_size

    _save_previews(image_path, raw_image, mask_selected, largest_obj, true_area_um2, true_perimeter_um, paths)
    _plot_images(raw_image, mask_selected)

    return {
        'Image': os.path.basename(image_path),
        'Perimeter (µm)': true_perimeter_um,
        'MajorAxisLength (µm)': major_axis_um,
        'Area (µm²)': true_area_um2,
        'Eccentricity': largest_obj.eccentricity
    }


def _save_previews(image_path, img, mask, props, true_area, true_perimeter, paths): 
    overlay = gray2rgb(img / img.max()) # Converts grayscale image to RGB for visualization
                                        # Normalizes values to 0–1 by dividing by the max intensity
    overlay[mask] = [1, 0, 0] # Colors the selected region as red

    # Prep image for cv2 drawing 
    preview = cv2.cvtColor((overlay * 255).astype(np.uint8), cv2.COLOR_RGB2BGR) # Converts  image from 0-1 floats to 0-255 integers (since it was a 8-bit image), then converts from RGB to BGR (OpenCV format)

    # Contour detection on padded mask of selected object only
    mask_float = mask.astype(float) # Converts boolean mask to floats as 'find_contours()' from skimage.measure expects a 2D float array, not a boolean array
    padded_mask = np.pad(mask_float, pad_width=1, mode='constant', constant_values=0) # Adds a 1-pixel padding border to prevent contour errors at the edges, so if the object touches the edge of the image, the contour won’t get clipped or broken.
    contours = find_contours(padded_mask, level=0.5) # Finds object boundaries at value 0.5 (i.e. edges between 0 and 1)
    # 0: background. 1: object. 0.5: in between

    for contour in contours:
        contour -= 1  #  Shifts contour back by 1 pixel (undoes padding: space between content + border)
        contour = np.flip(contour, axis=1)  # Flip (row, col) to (x, y)
        contour = np.round(contour).astype(np.int32) # Rounds + converts to integers
        cv2.polylines(preview, [contour], isClosed=True, color=(0, 255, 0), thickness=2) # draw a green contour on preview

    # Add text annotations
    y, x = props.centroid # Gets  coordinates of the center of the object 
    font = cv2.FONT_HERSHEY_TRIPLEX
    font_scale = 1.2 
    color = (255, 0, 0) # BLUE
    thickness = 2
    line_spacing = 40

    # Where to place the first line of text 
    start_x = int(x + 200)
    start_y = int(y - 1 * line_spacing)

    # Draws the things we need
    cv2.putText(preview, f"Perimeter: {true_perimeter:.2f}", (start_x, start_y), font, font_scale, color, thickness)
    cv2.putText(preview, f"Major Axis Length: {props.major_axis_length:.2f}", (start_x, start_y + line_spacing), font, font_scale, color, thickness)
    cv2.putText(preview, f"Area: {true_area:.2f}", (start_x, start_y + 2 * line_spacing), font, font_scale, color, thickness)
    cv2.putText(preview, f"Eccentricity: {props.eccentricity:.2f}", (start_x, start_y + 3 * line_spacing), font, font_scale, color, thickness)

    # Saves annotated preview image into a folder
    _save_image(preview, os.path.join(paths['rp'], os.path.basename(image_path)))

def _plot_images(image, mask):
    overlay = gray2rgb(image / image.max()) # Normalizes grayscale image so pixel values are between 0-1. Converts it to an RGB image so it can draw colored overlays on it
    overlay[mask] = [1, 0, 0] # Applies a red overlay

    """
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    axes[0].imshow(image, cmap='gray')
    axes[0].set_title("Original Image")
    axes[0].axis('off')

    axes[1].imshow(overlay)
    axes[1].set_title("Processed with the Mask")
    axes[1].axis('off')

    plt.tight_layout()
    plt.show()
    """

def _save_image(image, path):
    imsave(path, image)

results_list = []

for (image_path, raw_image, paths_map) in loaded_images:
    print(f"  → Processing: {os.path.relpath(image_path, input_root)}")
    result = size_qc(image_path, raw_image, paths_map, **params)
    if result:
        results_list.append(result)

 
# Save CSV if we have results
if results_list:
    results_df = pd.DataFrame(results_list)
    csv_path = os.path.join(results_base, "Results.csv")
    results_df.to_csv(csv_path, index=False)
    print(f"  Results saved: {csv_path}")
else:
    print("  No valid objects detected in this type folder.")